# Import 

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 248 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 32.1 MB/s 
     |████████████████████████████████| 9.9 MB 92.1 MB/s 
     |████████████████████████████████| 3.8 MB 87.5 MB/s 
     |████████████████████████████████| 1.7 MB 59.5 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 325 kB 59.0 MB/s 
     |████████████████████████████████| 6.5 MB 49.2 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
     |████████████████████████████████| 895 kB 66.8 MB/s 
     |████████████████████████████████| 67 kB 7.5 MB/s 
     |████████████████████████████████| 181 kB 73.6 MB/s 
     |████████████████████████████████| 144 kB 71.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 134 kB 60.9 MB/s 
     |████████████████████████████████| 212 kB 97.0 MB/s 
     |████████████████████████████████| 1.1 MB 70.6 MB/s 
     |███████████████

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
from matplotlib import pyplot as plt

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
from scipy.special import softmax
from google.colab import files

#Data prepration

##Read dataset

In [ ]:
tra_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/4192/Data/DirectCompare/train_df.csv")

tra_df = tra_df[['selftext','Expert-label']]
tra_df

,selftext,Expert-label
0,I posted this on Piazza but thought I might as...,1.0
1,"Hi i’ve applied for arts from Vancouver,BC as ...",0.0
2,i'm an international student and i've been tak...,1.0
3,i'm an international student and the midterm w...,1.0
4,they think i wouldnt be able to handle the str...,0.0
...,...,...
997,My boyfriend is Canadian and I’m American. Obv...,0.0
998,Do you need to be vaccinated to travel domesti...,0.0
999,"Hello, are there any International students he...",1.0
1000,Will you guys take a leave of absence? Or are ...,0.0


In [ ]:
tra_df = tra_df.iloc[:400]
tra_df

,selftext,Expert-label
0,I posted this on Piazza but thought I might as...,1.0
1,"Hi i’ve applied for arts from Vancouver,BC as ...",0.0
2,i'm an international student and i've been tak...,1.0
3,i'm an international student and the midterm w...,1.0
4,they think i wouldnt be able to handle the str...,0.0
...,...,...
395,"Hey guys, I think no one has a specific idea a...",1.0
396,Hey everyone\n\nI'm an international student c...,0.0
397,I'm an international student. Santa Ono posted...,1.0
398,I’m an international student and I was hoping ...,1.0


In [ ]:
#concat iteration result
it_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/4192/draft/Iteration/simualte_hrir/Iteration1/bert-base/b_b_val_i1.csv")

it_df = it_df[['selftext','Expert-label']]
it_df

,selftext,Expert-label
0,I'm an international student so I don't have a...,1.0
1,"Schools online, no tourists, and no internatio...",0.0
2,Do you think it would be okay to go back home ...,1.0
3,"Good morning everyone, I’m an international st...",0.0
4,I’m an international student and planning to g...,1.0
...,...,...
195,I'm a final year international student graduat...,1.0
196,"hello, \n\nI am an international student. \n\n...",0.0
197,How is it for international students living wi...,1.0
198,Any filipino/international student who’s takin...,0.0


In [ ]:
train_df = pd.concat([tra_df,it_df])
train_df

,selftext,Expert-label
0,I posted this on Piazza but thought I might as...,1.0
1,"Hi i’ve applied for arts from Vancouver,BC as ...",0.0
2,i'm an international student and i've been tak...,1.0
3,i'm an international student and the midterm w...,1.0
4,they think i wouldnt be able to handle the str...,0.0
...,...,...
195,I'm a final year international student graduat...,1.0
196,"hello, \n\nI am an international student. \n\n...",0.0
197,How is it for international students living wi...,1.0
198,Any filipino/international student who’s takin...,0.0


In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/4192/Data/Validation988/988validation.csv")

test_df = test_df[['selftext']]
test_df

,selftext
0,Well... I think I need some help... about depr...
1,I am an international student and i started en...
2,i'm an international student and the midterm w...
3,"Honestly just want to end it all , it’s so har..."
4,"Hi all, \n\nI'm really upset to know the Winte..."
...,...
983,WE NEED A SCIENCE AND DATA BASED APPROACH TO C...
984,Hi everyone! Hope that all who are applying ar...
985,Hello everyone !\n\nI request some advice from...
986,"Hi, I was wondering what my chances were at th..."


In [ ]:
test_data = test_df['selftext'].values.tolist()

In [ ]:
ret_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/4192/draft/Iteration/simualte_hrir/Iteration1/bert-base/b_b_i1_ret.csv")

ret_df = ret_df[['selftext']]
ret_df

,selftext
0,Any international students here travelling out...
1,Do you think that the start date for ryerson w...
2,Is all poli sci classes in-person? I received ...
3,University of Windsor vs mcmaster for mechani...
4,Can I transfer credit for undergraduate progra...
...,...
5601,Google doc version here: [https://docs.google....
5602,I am an international student who got accepted...
5603,"Hello, I'm a high school senior considering Wa..."
5604,I am an international student who got accepted...


In [ ]:
ret_data = ret_df['selftext'].values.tolist()

#Model

In [ ]:
model_args = ClassificationArgs(train_batch_size=4, gradient_accumulation_steps=16, learning_rate=3e-5, num_train_epochs= 3, max_seq_length= 512, overwrite_output_dir= True)

In [ ]:
model_bt = ClassificationModel('bert', 'bert-base-uncased',  args = model_args, use_cuda=True)
model_bt.train_model(train_df)
  

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

...

Running Epoch 0 of 3:   0%|          | 0/150 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/150 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/150 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


(27, 0.6795291194209346)

#Predict

##result

In [ ]:
result_ret,proba_ret = model_bt.predict(ret_data)
probabilities = softmax(proba_ret, axis=1)
proba_ret_df=pd.DataFrame(probabilities)
proba_ret_df.columns = ["remove","predict_result"]
proba_ret_df = proba_ret_df["predict_result"]
proba_ret_df

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5606 [00:00<?, ?it/s]

  0%|          | 0/701 [00:00<?, ?it/s]

0       0.705100
1       0.645380
2       0.562837
3       0.526586
4       0.629931
          ...   
5601    0.276739
5602    0.280616
5603    0.272266
5604    0.279042
5605    0.277424
Name: predict_result, Length: 5606, dtype: float64

In [ ]:
#Sorting

con_df = pd.concat([ret_df,proba_ret_df],axis=1)

con_df.sort_values(by='predict_result',axis=0,ascending=False,inplace=True)
b_b_i2 = con_df.iloc[:200]
b_b_i2

,selftext,predict_result
38,"Hello, are there any International students he...",0.752081
45,"Sorry if this is asked a lot, when will border...",0.748938
228,Hi everyone I'm wondering if there's any chanc...,0.747915
160,For international students in Canada would our...,0.745526
379,I am an international student and currently I’...,0.744029
...,...,...
62,I’m an international student considering livin...,0.686094
1162,With the recent announcement by the government...,0.685989
447,What about international students?,0.685825
351,https://twitter.com/Olufemiloye/status/138853...,0.685180


In [ ]:
#Saving to drive
b_b_i2.to_csv('b_b_i2.csv')
files.download('b_b_i2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
con_df.drop(con_df.head(200).index,inplace=True)
con_df

,selftext,predict_result
219,As an international student for Fall 21’ I wan...,0.685114
507,I am an international student from India. I ha...,0.685022
683,Hi I'm an international student and I was look...,0.684910
638,Hey guys! So I'm a returning student who has g...,0.684818
237,I’m a second year international student so I’m...,0.684788
...,...,...
5578,"Today, I saw a Piazza post on the CSC148 cours...",0.264406
5595,I managed a 3.25/5 GPA in my high school dip...,0.264026
5368,I would've never expected that I would be some...,0.263836
5551,I finished all of my own finals in the first h...,0.263268


In [ ]:
#Saving to drive
con_df.to_csv('b_b_i2_ret.csv')
files.download('b_b_i2_ret.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##988 recall

In [ ]:
result_bt,proba_bt = model_bt.predict(test_data)
result_bt_df = pd.DataFrame(result_bt)
result_bt_df.columns = ["predict_result"]
tp = result_bt_df.sum()
recall_res=tp/988

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/988 [00:00<?, ?it/s]

  0%|          | 0/124 [00:00<?, ?it/s]

In [ ]:
recall_res

predict_result    0.034413
dtype: float64